In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing
import sklearn.neural_network

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=ImportWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=ResourceWarning)

In [3]:
import theano
import theano.tensor as T

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [4]:
from keras.datasets import mnist

Using TensorFlow backend.


In [6]:
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

In [7]:
train_data.shape

(60000, 28, 28)

In [8]:
test_data.shape

(10000, 28, 28)

In [9]:
#reshape and standardize
x=train_data
y=test_data
xres=x.reshape(60000,-1)
yres=y.reshape(10000,-1)
xsd=sklearn.preprocessing.normalize(xres,axis=0)
ysd=sklearn.preprocessing.normalize(yres,axis=0)

In [12]:
xsd.shape

(60000, 784)

In [13]:
n_hidden = 100 # the size of hidden layers in MLP
n_latent = 2 # the dimension of z
n_input = train_data.shape[1] # the dimension of x's feature space

In [20]:
# Parameters
# Gaussian MLP weights and biases (encoder)
#initialize \phi (assume iterations = 10000)

b3 = np.zeros([10000, n_hidden])  # gaussian_bh = init_weights((n_hidden, ))
b3[0,] = np.random.normal(0,0.01,n_hidden)
b2 = np.zeros([10000, n_latent])  # sigma_bo = init_weights((n_latent, ))
b2[0,] = np.random.normal(0,0.01,n_latent)
b1 = np.zeros([10000, n_latent])  # mu_bo = init_weights((n_latent, ))
b1[0,] = np.random.normal(0,0.01,n_latent)

W3 = np.zeros([10000, n_input, n_hidden])  # gaussian_Wh = init_weights((n_input, n_hidden))
W3[0,] = np.random.normal(0,0.01,[n_input, n_hidden])
W2 = np.zeros([10000, n_hidden, n_latent])  # sigma_Wo = init_weights((n_hidden, n_latent))
W2[0,] = np.random.normal(0,0.01,[n_hidden, n_latent])
W1 = np.zeros([10000, n_hidden, n_latent])  # mu_Wo = init_weights((n_hidden, n_latent))
W1[0,] = np.random.normal(0,0.01,[n_hidden, n_latent])


In [ ]:
# Only the weight matrices W will be regularized (weight decay)
W = [gaussian_Wh, mu_Wo, sigma_Wo, bernoulli_Wh, bernoulli_Wo]
b = [gaussian_bh, mu_bo, sigma_bo, bernoulli_bh, bernoulli_bo]
params = W + b

In [24]:
# Gaussian Encoder
x = T.matrix("x")
h_encoder = T.tanh(T.dot(x, W3) + b3)

Elemwise{tanh,no_inplace}.0

In [ ]:
# Gaussian Encoder
x = T.matrix("x")
h_encoder = T.tanh(T.dot(x, gaussian_Wh) + gaussian_bh)
mu = T.dot(h_encoder, mu_Wo) + mu_bo
log_sigma = 0.5*(T.dot(h_encoder, sigma_Wo) + sigma_bo)
# This expression is simple (not an expectation) because we're using normal priors and posteriors
DKL = (1.0 + 2.0*log_sigma - mu**2 - T.exp(2.0*log_sigma)).sum(axis = 1)/2.0

In [ ]:
# Lower bound
lower_bound = -(DKL + log_likelihood).mean()
# Weight decay
L2 = sum([(w**2).sum() for w in W])
cost = lower_bound + batch_size/50000.0/2.0*L2

In [ ]:
def adagrad(cost, params, lr=0.001, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = acc + g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

In [ ]:
updates = adagrad(lower_bound, params, lr=0.02)

In [ ]:
train_model = theano.function(inputs=[x, std_normal], 
                              outputs=cost, 
                              updates=updates,
                              mode='FAST_RUN',
                              allow_input_downcast=True)
                              
eval_model = theano.function(inputs=[x, std_normal], outputs=lower_bound,
                             mode='FAST_RUN',
                             allow_input_downcast=True) 